# 1. IMPORTS

In [89]:
import os,pandas,sys,time,keras,json,sklearn,tensorflow,random
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ['PYTHONHASHSEED']='0'

def reseed():
    seed = 0x5f3759df
    np.random.seed(seed)
    tensorflow.random.set_seed(seed)
    keras.utils.set_random_seed(seed)
    random.seed(seed)

from starter import Starter
starter = Starter()
starter.start(lambda: os.chdir(os.path.dirname(os.getcwd())))

Starter has already been initialized.


# 2. LOAD DATA

In [90]:
#!/usr/bin/env python3
from utils.data_loader import DataLoader
import numpy as np 

print("1. LOADING AND PREPROCESSING DATA")
print("="*50)

# Initialize data loader
data_dir = "../data" 
data_loader = DataLoader(data_dir)

# Load and prepare data
try:
    X_train, y_train, X_valid, y_valid, X_test, y_test = data_loader.prepare_data(
        max_vocab_size=5000,
        max_length=50,
        min_freq=1
    )
    
    print("\nData loaded successfully!")
    print(f"Vocabulary size: {data_loader.preprocessor.vocab_size}")
    print(f"Number of classes: {data_loader.num_classes}")
    print(f"Max sequence length: {data_loader.preprocessor.max_length}")
    
    # Show class distribution
    unique, counts = np.unique(y_train, return_counts=True)
    print("\nClass distribution in training data:")
    for class_id, count in zip(unique, counts):
        class_name = data_loader.reverse_label_encoder[class_id]
        print(f"  {class_name}: {count} ({count/len(y_train)*100:.1f}%)")
        
    print("\nData shapes:")
    print(f"  Training: X={X_train.shape}, y={y_train.shape}")
    print(f"  Validation: X={X_valid.shape}, y={y_valid.shape}")
    print(f"  Test: X={X_test.shape}, y={y_test.shape}")
    
    # Display sample data
    print("\nSample data:")
    print(f"  First training text tokens: {X_train[0][:10]}...")
    print(f"  First training label: {y_train[0]} ({data_loader.reverse_label_encoder[y_train[0]]})")
    
    # Verify data integrity
    print("\nData integrity checks:")
    print(f"  No missing values in X_train: {not np.any(np.isnan(X_train))}")
    print(f"  No missing values in y_train: {not np.any(np.isnan(y_train))}")
    print(f"  All labels in valid range: {np.all((y_train >= 0) & (y_train < data_loader.num_classes))}")
    
    print("\nData preprocessing completed successfully!")
        
except Exception as e:
    print(f"Error loading data: {e}")
    import traceback
    traceback.print_exc()
    raise e


1. LOADING AND PREPROCESSING DATA
Loaded data:
  Train: 500 samples
  Valid: 100 samples
  Test: 400 samples
Built vocabulary with 2796 words
Most frequent words: ['yang', 'di', 'dan', 'saya', 'tidak', 'dengan', 'enak', 'ini', 'makan', 'untuk']
Label encoding:
  negative: 0
  neutral: 1
  positive: 2

Data shapes:
  X_train: (500, 50)
  y_train: (500,)
  X_valid: (100, 50)
  y_valid: (100,)
  X_test: (400, 50)
  y_test: (400,)

Data loaded successfully!
Vocabulary size: 2796
Number of classes: 3
Max sequence length: 50

Class distribution in training data:
  negative: 192 (38.4%)
  neutral: 119 (23.8%)
  positive: 189 (37.8%)

Data shapes:
  Training: X=(500, 50), y=(500,)
  Validation: X=(100, 50), y=(100,)
  Test: X=(400, 50), y=(400,)

Sample data:
  First training text tokens: [1152  736  737  183 1153  184   11  554  185  738]...
  First training label: 1 (neutral)

Data integrity checks:
  No missing values in X_train: True
  No missing values in y_train: True
  All labels in val

# 3. INITIALIZE EXPERIMENT RUNNER

In [91]:
#!/usr/bin/env python3
import os
import time
import json
import numpy as np
import keras
import sklearn.metrics


class KerasLSTMExperiment:
    """Keras LSTM experiment class for systematic hyperparameter analysis"""
    
    def __init__(self, data_loader, X_train, y_train, X_valid, y_valid, X_test, y_test):
        self.data_loader = data_loader
        self.X_train = X_train
        self.y_train = y_train
        self.X_valid = X_valid
        self.y_valid = y_valid
        self.X_test = X_test
        self.y_test = y_test
        
        # Base configuration
        self.base_config = {
            'vocab_size': data_loader.preprocessor.vocab_size,
            'embedding_dim': 64,
            'lstm_units': 32,
            'num_classes': data_loader.num_classes,
            'max_length': data_loader.preprocessor.max_length,
            'activation': 'tanh',
            'dropout_rate': 0.2,
            'learning_rate': 0.001,
            'batch_size': 32,
            'epochs': 15
        }
        
        print(f"Base configuration:")
        for key, value in self.base_config.items():
            print(f"  {key}: {value}")
    
    def create_keras_model(self, config):
        """Create Keras lstm model with given configuration"""
        reseed()
        model = keras.models.Sequential()
        model.add(keras.layers.Embedding(
            input_dim=config['vocab_size'],
            output_dim=config['embedding_dim'],
            input_length=config['max_length'],
            name='embedding'
        ))
        
        for i in range(config['num_lstm_layers']):
            return_sequences = i < config['num_lstm_layers'] - 1
            lstm_layer = keras.layers.LSTM(
                units=config['lstm_units'],
                activation=config['activation'],
                return_sequences=return_sequences,
                name=f'lstm_{i}'
            )
            
            if config['bidirectional']:
                model.add(keras.layers.Bidirectional(lstm_layer, name=f'bidirectional_lstm_{i}'))
            else:
                model.add(lstm_layer)
            
            if i < config['num_lstm_layers'] - 1:
                model.add(keras.layers.Dropout(config['dropout_rate'], name=f'dropout_{i}'))
        
        model.add(keras.layers.Dropout(config['dropout_rate'], name='dropout_final'))
        model.add(keras.layers.Dense(config['num_classes'], activation='softmax', name='classification'))
        
        model.build(input_shape=(None, config['max_length']))
        
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=config['learning_rate']),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        
        return model
    
    def train_and_evaluate(self, config, experiment_name):
        print(f"\n{'='*60}")
        print(f"Training: {experiment_name}")
        print(f"Config: {config}")
        print(f"{'='*60}")
        
        start_time = time.time()
        model = self.create_keras_model(config)
        print(f"Model created with {model.count_params():,} parameters")
        
        print(f"Starting training for {config['epochs']} epochs...")
        history = model.fit(
            self.X_train, self.y_train,
            validation_data=(self.X_valid, self.y_valid),
            epochs=config['epochs'],
            batch_size=config['batch_size'],
            verbose=1,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True
                )
            ],
            shuffle=False
        )
        
        print(f"Evaluating on test set...")
        test_loss, test_acc = model.evaluate(self.X_test, self.y_test, verbose=0)
        test_predictions = model.predict(self.X_test, verbose=0)
        test_pred_classes = np.argmax(test_predictions, axis=1)
        test_f1_macro = sklearn.metrics.f1_score(self.y_test, test_pred_classes, average='macro')
        
        valid_predictions = model.predict(self.X_valid, verbose=0)
        valid_pred_classes = np.argmax(valid_predictions, axis=1)
        valid_f1_macro = sklearn.metrics.f1_score(self.y_valid, valid_pred_classes, average='macro')
        
        weights_path = f"results/{experiment_name}_weights.npz"
        os.makedirs("results", exist_ok=True)
        self.save_keras_weights(model, weights_path, config)

        unique_rows = np.unique(self.X_test, axis=0)

        
        training_time = time.time() - start_time
        
        print(f"\nResults for {experiment_name}:")
        print(f"  Training time: {training_time:.2f} seconds")
        print(f"  Test Accuracy: {test_acc:.4f}")
        print(f"  Test F1-Score (macro): {test_f1_macro:.4f}")
        print(f"  Valid F1-Score (macro): {valid_f1_macro:.4f}")
        print(f"  Weights saved to: {weights_path}")
        
        return {
            'model': model,
            'history': history.history,
            'test_accuracy': test_acc,
            'test_f1_score': test_f1_macro,
            'valid_f1_score': valid_f1_macro,
            'weights_path': weights_path,
            'config': config,
            'training_time': training_time
        }
    
    def save_keras_weights(self, model, filepath, config):
        print(f"Saving Keras weights to: {filepath}")
        
        try:
            if len(model.weights) == 0:
                raise ValueError("Model has no weights to save!")
            
            weights_dict = {}
            lstm_layer_count = 0
            
            for layer in model.layers:
                layer_weights = layer.get_weights()
                if len(layer_weights) == 0:
                    continue
                    
                layer_name = layer.name
                print(f"  Processing layer: {layer_name} - {len(layer_weights)} weight arrays")
                
                if 'embedding' in layer_name.lower():
                    weights_dict['embedding'] = {
                        'embedding_matrix': layer_weights[0]
                    }
                elif 'simple_lstm' in layer_name.lower():
                    target_name = f'lstm_{lstm_layer_count}'
                    weights_dict[target_name] = {
                        'W_ih': layer_weights[0].T,
                        'W_hh': layer_weights[1].T,
                        'b_h': layer_weights[2]
                    }
                    lstm_layer_count += 1
                elif 'bidirectional' in layer_name.lower():
                    target_name = f'bidirectional_lstm_{lstm_layer_count}'
                    if len(layer_weights) >= 6:
                        weights_dict[target_name] = {
                            'forward_W_ih': layer_weights[0].T,
                            'forward_W_hh': layer_weights[1].T,
                            'forward_b_h': layer_weights[2],
                            'backward_W_ih': layer_weights[3].T,
                            'backward_W_hh': layer_weights[4].T,
                            'backward_b_h': layer_weights[5]
                        }
                    lstm_layer_count += 1
                elif 'dense' in layer_name.lower() or 'classification' in layer_name.lower():
                    weights_dict['classification'] = {
                        'W': layer_weights[0].T,
                        'b': layer_weights[1]
                    }
            
            save_dict = {}
            for layer_name, layer_weights in weights_dict.items():
                for weight_name, weight_value in layer_weights.items():
                    save_dict[f"{layer_name}_{weight_name}"] = weight_value
            
            save_dict['config'] = json.dumps(config)
            np.savez(filepath, **save_dict)
            print(f"  Saved {len(save_dict)-1} weight arrays successfully")
            print(f"  Layers saved: {list(weights_dict.keys())}")
            
            loaded_check = np.load(filepath)
            assert len(loaded_check.files) == len(save_dict), "Save verification failed"
            loaded_check.close()
        
        except Exception as e:
            print(f"Error saving weights: {e}")
            import traceback
            traceback.print_exc()
            raise


print("Initializing Keras experiment framework...")
keras_experiment = KerasLSTMExperiment(
    data_loader, X_train, y_train, X_valid, y_valid, X_test, y_test
)
print("Keras experiment runner initialized!")


Initializing Keras experiment framework...
Base configuration:
  vocab_size: 2796
  embedding_dim: 64
  lstm_units: 32
  num_classes: 3
  max_length: 50
  activation: tanh
  dropout_rate: 0.2
  learning_rate: 0.001
  batch_size: 32
  epochs: 15
Keras experiment runner initialized!


# 4. EXPERIMENTS

## 4.1. Comparing Layer Counts

In [ ]:
#!/usr/bin/env python3
layer_counts = [1, 3, 5]
layer_results = {}

for num_layers in layer_counts:
    config = keras_experiment.base_config.copy()
    config.update({
        'num_lstm_layers': num_layers,
        'bidirectional': False, 
        'lstm_units': 64
    })
    # config['epochs'] = 10
    
    experiment_name = f"lstm_layers_{num_layers}"
    print(f"\n🔬 Experiment: {experiment_name}")
    print(f"   Layers: {num_layers}, Units: {config['lstm_units']}, Bidirectional: {config['bidirectional']}")
    
    try:
        result = keras_experiment.train_and_evaluate(config, experiment_name)
        for k,v in result.items():
            print(k, v)
        layer_results[num_layers] = result
        print(f" {experiment_name} completed successfully!")
    except Exception as e:
        print(f" Error in {experiment_name}: {e}")
        continue

print(f"{'Layers':<8} {'Accuracy':<10} {'F1-Score':<10} {'Time (s)':<10}")
print("-"*40)
for layers in sorted(layer_results.keys()):
    result = layer_results[layers]
    print(f"{layers:<8} {result['test_accuracy']:.4f}    {result['test_f1_score']:.4f}    {result['training_time']:.1f}")



🔬 Experiment: lstm_layers_1
   Layers: 1, Units: 64, Bidirectional: False

Training: lstm_layers_1
Config: {'vocab_size': 2796, 'embedding_dim': 64, 'lstm_units': 64, 'num_classes': 3, 'max_length': 50, 'activation': 'tanh', 'dropout_rate': 0.2, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 15, 'num_lstm_layers': 1, 'bidirectional': False}
Model created with 212,163 parameters
Starting training for 15 epochs...
Epoch 1/15


/mnt/d/GitHub/tugas-besar-ml-2/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.3653 - loss: 1.0973 - val_accuracy: 0.3800 - val_loss: 1.0841
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4049 - loss: 1.0877 - val_accuracy: 0.3900 - val_loss: 1.0789
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.4241 - loss: 1.0724 - val_accuracy: 0.5100 - val_loss: 0.9840
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4915 - loss: 0.9597 - val_accuracy: 0.5100 - val_loss: 0.9718
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5670 - loss: 0.8813 - val_accuracy: 0.5400 - val_loss: 0.8799
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6365 - loss: 0.7261 - val_accuracy: 0.6100 - val_loss: 0.8942
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7185 - loss: 0.5596 - val_accuracy: 0.5900 - val_loss: 0.9539
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.7327 - loss: 0.5469 - val_accuracy: 0.6100 - val_loss: 0.

/mnt/d/GitHub/tugas-besar-ml-2/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 111ms/step - accuracy: 0.4096 - loss: 1.0935 - val_accuracy: 0.3800 - val_loss: 1.0805
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.3632 - loss: 1.0674 - val_accuracy: 0.4800 - val_loss: 1.0184
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.5599 - loss: 0.9044 - val_accuracy: 0.5300 - val_loss: 0.9779
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.6679 - loss: 0.7597 - val_accuracy: 0.6800 - val_loss: 0.8524
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.8056 - loss: 0.5422 - val_accuracy: 0.6100 - val_loss: 0.9270
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.8491 - loss: 0.3842 - val_accuracy: 0.6800 - val_loss: 0.9278
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 0.8705 - loss: 0.4158 - val_accuracy: 0.5300 - val_loss: 1.2673
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.7605 - loss: 0.6470 - val_accuracy: 0.6300 - val_loss: 0

/mnt/d/GitHub/tugas-besar-ml-2/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


 5/16 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.4044 - loss: 1.0961

## 4.2. Comparing Cell/Unit Counts

In [ ]:
#!/usr/bin/env python3
cell_counts = [16, 32, 64, 128, 256]
cell_results = {}

for num_cells in cell_counts:
    config = keras_experiment.base_config.copy()
    config.update({
        'num_lstm_layers': 1,
        'bidirectional': False, 
        'lstm_units': num_cells
    })
    experiment_name = f"lstm_cells_{num_cells}"
    
    try:
        result = keras_experiment.train_and_evaluate(config, experiment_name)
        cell_results[num_cells] = result
        print(f" {experiment_name} completed successfully!")
    except Exception as e:
        print(f" Error in {experiment_name}: {e}")
        continue

print(f"{'Cells':<8} {'Accuracy':<10} {'F1-Score':<10} {'Time (s)':<10}")
print("-"*40)
for cells in sorted(cell_results.keys()):
    result = cell_results[cells]
    print(f"{cells:<8} {result['test_accuracy']:.4f}    {result['test_f1_score']:.4f}    {result['training_time']:.1f}")



Training: lstm_cells_16
Config: {'vocab_size': 2796, 'embedding_dim': 64, 'lstm_units': 16, 'num_classes': 3, 'max_length': 50, 'activation': 'tanh', 'dropout_rate': 0.2, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 15, 'num_lstm_layers': 1, 'bidirectional': False}
Model created with 184,179 parameters
Starting training for 15 epochs...
Epoch 1/15


/mnt/d/GitHub/tugas-besar-ml-2/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 42ms/step - accuracy: 0.3617 - loss: 1.0964 - val_accuracy: 0.4000 - val_loss: 1.0844
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4070 - loss: 1.0882 - val_accuracy: 0.4000 - val_loss: 1.0829
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4398 - loss: 1.0816 - val_accuracy: 0.3800 - val_loss: 1.0808
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4489 - loss: 1.0711 - val_accuracy: 0.3900 - val_loss: 1.0706
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5246 - loss: 1.0235 - val_accuracy: 0.5800 - val_loss: 0.8838
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6847 - loss: 0.7358 - val_accuracy: 0.6200 - val_loss: 0.8477
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7282 - loss: 0.6083 - val_accuracy: 0.5900 - val_loss: 0.8954
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7297 - loss: 0.5837 - val_accuracy: 0.5500 - val_loss: 1.

/mnt/d/GitHub/tugas-besar-ml-2/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.3929 - loss: 1.0941 - val_accuracy: 0.3800 - val_loss: 1.0844
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.4036 - loss: 1.0874 - val_accuracy: 0.3700 - val_loss: 1.0821
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.4085 - loss: 1.0848 - val_accuracy: 0.3700 - val_loss: 1.0775
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4319 - loss: 1.0710 - val_accuracy: 0.4900 - val_loss: 1.0266
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.4913 - loss: 0.9565 - val_accuracy: 0.5700 - val_loss: 0.9216
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6298 - loss: 0.7412 - val_accuracy: 0.5900 - val_loss: 0.8588
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6914 - loss: 0.6338 - val_accuracy: 0.5400 - val_loss: 0.9877
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6948 - loss: 0.6258 - val_accuracy: 0.6000 - val_loss: 1.

...

16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 125ms/step - accuracy: 0.3653 - loss: 1.0973 - val_accuracy: 0.3800 - val_loss: 1.0841
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.4049 - loss: 1.0877 - val_accuracy: 0.3900 - val_loss: 1.0789
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.4241 - loss: 1.0724 - val_accuracy: 0.5100 - val_loss: 0.9840
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.4915 - loss: 0.9597 - val_accuracy: 0.5100 - val_loss: 0.9718
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.5670 - loss: 0.8813 - val_accuracy: 0.5400 - val_loss: 0.8799
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.6365 - loss: 0.7261 - val_accuracy: 0.6100 - val_loss: 0.8942
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.7185 - loss: 0.5596 - val_accuracy: 0.5900 - val_loss: 0.9539
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.7327 - loss: 0.5469 - val_accuracy: 0.6100 - val_loss: 0

/mnt/d/GitHub/tugas-besar-ml-2/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.3768 - loss: 1.0938 - val_accuracy: 0.3800 - val_loss: 1.0846
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.3768 - loss: 1.0872 - val_accuracy: 0.3700 - val_loss: 1.0674
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.4493 - loss: 1.0261 - val_accuracy: 0.5200 - val_loss: 0.9299
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.5828 - loss: 0.8519 - val_accuracy: 0.6000 - val_loss: 0.8626
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.6750 - loss: 0.7193 - val_accuracy: 0.6200 - val_loss: 0.8660
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.6945 - loss: 0.6116 - val_accuracy: 0.6000 - val_loss: 0.8793
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.7221 - loss: 0.5411 - val_accuracy: 0.5500 - val_loss: 1.0417
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.7116 - loss: 0.5737 - val_accuracy: 0.5500 - val_loss: 1.

/mnt/d/GitHub/tugas-besar-ml-2/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - accuracy: 0.3890 - loss: 1.0948 - val_accuracy: 0.3800 - val_loss: 1.0866
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - accuracy: 0.3736 - loss: 1.0828 - val_accuracy: 0.4500 - val_loss: 1.0370
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step - accuracy: 0.4696 - loss: 0.9874 - val_accuracy: 0.4800 - val_loss: 1.0257
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 159ms/step - accuracy: 0.5242 - loss: 0.9134 - val_accuracy: 0.5500 - val_loss: 0.9074
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 145ms/step - accuracy: 0.6642 - loss: 0.7012 - val_accuracy: 0.5700 - val_loss: 0.9177
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 141ms/step - accuracy: 0.6665 - loss: 0.6219 - val_accuracy: 0.5500 - val_loss: 1.0768
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 143ms/step - accuracy: 0.7141 - loss: 0.5987 - val_accuracy: 0.6300 - val_loss: 0.9587
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 147ms/step - accuracy: 0.6959 - loss: 0.6206 - val_accuracy: 0.6100 - val_

## 4.3. Comparing Bidirectional vs. Unidirectional Layers

In [ ]:
#!/usr/bin/env python3
ltype_results = {}
ltype_vars = [False, True]

for ltype in ltype_vars:
    config = keras_experiment.base_config.copy()
    config.update({
        'num_lstm_layers': 1,
        'bidirectional': ltype, 
        'lstm_units': 64
    })
    experiment_name = f"lstm_ltypes_{'bidirectional' if ltype else 'not unidirectional'}"
    
    try:
        result = keras_experiment.train_and_evaluate(config, experiment_name)
        ltype_results[ltype] = result
        print(f" {experiment_name} completed successfully!")
    except Exception as e:
        print(f" Error in {experiment_name}: {e}")
        continue

print(f"{'ltypes':<8} {'Accuracy':<10} {'F1-Score':<10} {'Time (s)':<10}")
print("-"*40)
for ltypes in sorted(ltype_results.keys()):
    result = ltype_results[ltypes]
    print(f"{ltypes:<8} {result['test_accuracy']:.4f}    {result['test_f1_score']:.4f}    {result['training_time']:.1f}")



Training: lstm_ltypes_not unidirectional
Config: {'vocab_size': 2796, 'embedding_dim': 64, 'lstm_units': 64, 'num_classes': 3, 'max_length': 50, 'activation': 'tanh', 'dropout_rate': 0.2, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 15, 'num_lstm_layers': 1, 'bidirectional': False}
Model created with 212,163 parameters
Starting training for 15 epochs...
Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - accuracy: 0.3653 - loss: 1.0973 - val_accuracy: 0.3800 - val_loss: 1.0841
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.4049 - loss: 1.0877 - val_accuracy: 0.3900 - val_loss: 1.0789
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.4241 - loss: 1.0724 - val_accuracy: 0.5100 - val_loss: 0.9840
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.4915 - loss: 0.9597 - val_accuracy: 0.5100 - val_loss: 0.9718
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.5670 - loss: 0.8813 - val_accuracy: 0.5400 - val_loss: 0.8799
E

/mnt/d/GitHub/tugas-besar-ml-2/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


KeyboardInterrupt: 